In [247]:
import json
import pandas as pd

In [248]:
!pip install psycopg2

# Opening JSON file 

In [249]:
with open("test.json") as datafile:
    data_test = json.load(datafile)

In [250]:
with open("train.json") as datafile:
    data_train = json.load(datafile)

In [251]:
data_train[0]

{'id': 10259,
 'cuisine': 'greek',
 'ingredients': ['romaine lettuce',
  'black olives',
  'grape tomatoes',
  'garlic',
  'pepper',
  'purple onion',
  'seasoning',
  'garbanzo beans',
  'feta cheese crumbles']}

In [252]:
data=pd.DataFrame(data_train[0])

In [253]:
data.index.values

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [254]:
data['cuisine'].values[0]

'greek'

In [255]:
data['id'].values[0]

10259

In [256]:
ingred_id=[]
cuisine_id=[]
cuisine=[]
dish_id=[]
ingredients=[]

for i in range(len(data_train)):
    data=pd.DataFrame(data_train[i])
    dish_id.append(i)
    for j in range(len(data)):
        ingred_id.append(data.index.values[j].astype(float))
        ingredients.append(data['ingredients'][j])
    cuisine.append(data['cuisine'].values[0])
    cuisine_id.append(data['id'].values[0].astype(float))    

In [257]:
len(data_train)

39774

In [258]:
import psycopg2

In [259]:
conn=psycopg2.connect(host='database-3.cj7bqpy1ed9k.us-east-2.rds.amazonaws.com', user='postgres', password='postgres', database='Recipe')

In [260]:
mycursor=conn.cursor()

In [261]:
conn.set_session(autocommit=True)

In [265]:
#creating a function merge which will merge two lists into a single list

def merge(list1, list2): 
      
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))] 
    return merged_list

In [266]:
record_dish=merge(dish_id,cuisine)

In [267]:
len(record_dish)

39774

In [268]:
type(record_dish)

list

In [269]:
import psycopg2.extras

In [270]:
#creating the table

try:
    mycursor.execute('CREATE TABLE ITEMS (DISH_ID INTEGER PRIMARY KEY, CUISINE_TYPE VARCHAR) ')
except psycopg2.errors as e:
    print(e)

In [271]:
#insert the 'record_dish' list to database 'items' one by one 

def insert_dish(data):
    try:
        sql_insert_query = """ INSERT INTO ITEMS (DISH_ID,CUISINE_TYPE) VALUES %s """
        psycopg2.extras.execute_values (mycursor, sql_insert_query, data, template=None, page_size=10000)
        _ = conn.commit()
    except psycopg2.errors as e:
        print(e)

In [272]:
insert_dish(record_dish)

In [273]:
record_cuisine=merge(cuisine_id,cuisine)

In [274]:
#creating the table

try:
    mycursor.execute('CREATE TABLE CUISINES (CUISINE_ID INTEGER PRIMARY KEY, CUISINE_NAME VARCHAR) ')
except psycopg2.errors as e:
    print(e)

In [275]:
#insert the 'record_cuisine' list to database 'cuisines' one by one 

def insert_cuisine(data):
    try:
        sql_insert_query = """ INSERT INTO CUISINES (CUISINE_ID,CUISINE_NAME) VALUES %s """
        psycopg2.extras.execute_values (mycursor, sql_insert_query, data, template=None, page_size=10000)
        _ = conn.commit()
    except psycopg2.errors as e:
        print(e)

In [276]:
insert_cuisine(record_cuisine)

In [281]:
#creating the table

try:
    mycursor.execute('CREATE TABLE INGREDIENTS (INGRED_ID INTEGER, INGREDIENTS VARCHAR) ')
except psycopg2.errors as e:
    print(e)

In [282]:
record_ingred=merge(ingred_id,ingredients)

In [283]:
#insert the 'record_ingred' list to database 'ingredients' one by one 

def insert_ingred(data):
    try:
        sql_insert_query = """ INSERT INTO INGREDIENTS (INGRED_ID,INGREDIENTS) VALUES %s """
        psycopg2.extras.execute_values (mycursor, sql_insert_query, data, template=None, page_size=10000)
        _ = conn.commit()
    except psycopg2.errors as e:
        print(e)

In [284]:
insert_ingred(record_ingred)

In [286]:
#creating the table

try:
    mycursor.execute('CREATE TABLE DISHES (DISH_ID INTEGER PRIMARY KEY,INGRED_ID INTEGER ) ')
except psycopg2.errors as e:
    print(e)

In [287]:
record_merge=merge(dish_id,ingred_id)

In [288]:
#insert the 'record_merge' list to database 'dishes' one by one 

def insert_merge(data):
    try:
        sql_insert_query = """ INSERT INTO DISHES (DISH_ID,INGRED_ID) VALUES %s """
        psycopg2.extras.execute_values (mycursor, sql_insert_query, data, template=None, page_size=10000)
        _ = conn.commit()
    except psycopg2.errors as e:
        print(e)

In [289]:
insert_merge(record_merge)